In [ ]:
from random import randint
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
dir_path = os.path.join(os.getcwd(), './CMAPSSData')
print(os.listdir(dir_path))

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
index_names = ['unit_nr', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['s_{}'.format(i) for i in range(1,22)] 
col_names = index_names + setting_names + sensor_names

In [ ]:
train = pd.read_csv((dir_path+'/train_FD001.txt'), sep='\s+', header=None, names=col_names)
test = pd.read_csv((dir_path+'/test_FD001.txt'), sep='\s+', header=None, names=col_names)
y_test = pd.read_csv((dir_path+'/RUL_FD001.txt'), sep='\s+', header=None, names=['RUL'])

In [ ]:
train

In [ ]:
train[index_names].describe()

According to the data, it is seen that the dataset contains simulations on 100 turbofan engines. 

In [ ]:
train[index_names].groupby('unit_nr').max().describe()

When inspecting the max time_cycles you can see the engine which failed the earliest did so after 128 cycles, whereas the engine which operated the longest broke down after 362 cycles. The average engine breaks between 199 and 206 cycles, however the standard deviation of 46 cycles is rather big. The standard deviation means the space of failure between any two different engines.

In [ ]:
train[setting_names].describe()

In [ ]:
train[sensor_names].describe().transpose()

By looking at the standard deviation it’s clear sensors 1, 10, 18 and 19 do not fluctuate at all, these can be safely discarded as they hold no useful information. Inspecting the quantiles indicates sensors 5, 6 and 16 have little fluctuation and require further inspection. Sensors 9 and 14 have the highest fluctuation, however this does not mean the other sensors can’t hold valuable information.

Now that we understand the data distribution of our data, we can compute the RUL (Remaining Useful Life) of the engine, which will give us an insight into how the sensor changes as the engine nears breakdown, which will also serve as the target variable of our work.

However there is no set predefined way to compute the RUL for our turbofan engines one way to do that is by assuming the RUL decreases linearly over time and have a value of 0 at the last time cycle of the engine. This assumption implies RUL would be 10 at 10 cycles before breakdown, 50 at 50 cycles before breakdown, etc.

In [ ]:
grouped_by_unit = train.groupby(by="unit_nr")
max_cycle = grouped_by_unit["time_cycles"].max()
    
    # Merge the max cycle back into the original frame
result_frame = train.merge(max_cycle.to_frame(name='max_cycle'), left_on='unit_nr', right_index=True)
    
    # Calculate remaining useful life for each row
remaining_useful_life = result_frame["max_cycle"] - result_frame["time_cycles"]
result_frame["RUL"] = remaining_useful_life

In [ ]:
result_frame = result_frame.drop("max_cycle", axis=1)

In [ ]:
result_frame

In [ ]:
train = result_frame.copy()

In [ ]:
train[index_names+['RUL']].head()

In [ ]:
dataset_description = train.describe()
print(dataset_description)

In [ ]:
axes = dataset_description.T.plot.bar(subplots=True, figsize=(15,10))

In [ ]:
data_corr = train.corr()
data_corr['RUL'].sort_values(ascending=False)

In [ ]:
df_plot = train.copy()[setting_names + sensor_names]
df_corr = df_plot.corr(method='pearson')
fig, ax = plt.subplots(figsize=(15,15))
axes = sns.heatmap(df_corr, linewidths=.2, )

In [ ]:
df_plot = train.copy()
df_plot = df_plot.sort_values(index_names)
graph = sns.PairGrid(data=df_plot, x_vars="RUL", y_vars=setting_names + sensor_names, hue="unit_nr", height=4, aspect=6,)
graph = graph.map(plt.plot, alpha=0.5)
graph = graph.set(xlim=(df_plot['RUL'].max(),df_plot['RUL'].min()))
# graph = graph.add_legend()

In [ ]:
def evaluate(y_true, y_hat, label='test'):
    mse = mean_squared_error(y_true, y_hat)
    rmse = np.sqrt(mse)
    variance = r2_score(y_true, y_hat)
    return '{} set RMSE:{}, R2:{}'.format(label, rmse, variance)

In [ ]:
drop_sensors = ['s_1','s_5','s_6','s_10','s_16','s_18','s_19']
drop_labels = index_names+setting_names+drop_sensors
X_train = train.drop(drop_labels, axis=1)
y_train = X_train.pop('RUL')

In [ ]:
test

In [ ]:
X_test = test.groupby('unit_nr').last().reset_index().drop(drop_labels, axis=1)

In [ ]:
X_test

In [ ]:
def model(X_train, y_train, X_test, y_test):
    from sklearn.linear_model import LinearRegression
    lin_reg = LinearRegression()
    lin_reg.fit(X_train, y_train)
    train_pred_lin = lin_reg.predict(X_train)
    test_pred_lin = lin_reg.predict(X_test)
    result_train = evaluate(y_train, train_pred_lin, 'train')
    result_test = evaluate(y_train, train_pred_lin)
    
    from sklearn.tree import DecisionTreeRegressor
    dec_tree = DecisionTreeRegressor()
    dec_tree.fit(X_train, y_train)
    train_pred_dec = dec_tree.predict(X_train)
    test_pred_dec = dec_tree.predict(X_test)
    
    from sklearn.ensemble import RandomForestRegressor
    ran_forest = RandomForestRegressor()
    ran_forest.fit(X_train, y_train)
    train_pred_ran = ran_forest.predict(X_train)
    test_pred_ran = ran_forest.predict(X_test)
    
    from sklearn.svm import SVR
    svr = SVR(kernel='linear')
    svr.fit(X_train, y_train)
    train_pred_svr = svr.predict(X_train)
    test_pred_svr = svr.predict(X_test)
    
    from sklearn.linear_model import LassoCV
    lasso = LassoCV()
    lasso.fit(X_train, y_train)
    train_pred_lasso = lasso.predict(X_train)
    test_pred_lasso = lasso.predict(X_test)

    
    print('Linear Regression Results \n', evaluate(y_train, train_pred_lin, 'train'), '\n', evaluate(y_test, test_pred_lin), '\n')
    print('Decision Tree Regression Results \n', evaluate(y_train, train_pred_dec, 'train'), '\n', evaluate(y_test, test_pred_dec), '\n')
    print('Random Forest Regression Results \n', evaluate(y_train, train_pred_ran, 'train'), '\n', evaluate(y_test, test_pred_ran), '\n')
    print('Support Vector Regression Results \n', evaluate(y_train, train_pred_svr, 'train'), '\n', evaluate(y_test, test_pred_svr), '\n')
    print('Lasso Regression Results \n', evaluate(y_train, train_pred_lasso, 'train'), '\n', evaluate(y_test, test_pred_lasso), '\n')

In [ ]:
model(X_train, y_train, X_test, y_test)

In [ ]:
#Performing some feature engineering
y_train_clipped = y_train.clip(upper=125)

In [ ]:
model(X_train, y_train_clipped, X_test, y_test)

In [ ]:
#Scaling our dataset
# Scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model(X_train_scaled, y_train_clipped, X_test_scaled, y_test)